In [ ]:

from autumn.core.project import get_project, load_timeseries
from autumn.settings import Region, Models
from matplotlib import pyplot
from autumn.core.plots.utils import REF_DATE
import pandas as pd
from autumn.core.plots.uncertainty.plots import _get_target_values, _plot_targets_to_axis
from summer.utils import ref_times_to_dti
from autumn.core.plots.utils import REF_DATE

In [ ]:
# load project
project = get_project(Models.SM_COVID, Region.PHILIPPINES)

In [ ]:
# overwrite default parameters with calibrated parameters
calibrated_params = {
    #  list all the calibrated parameters here
    "contact_rate": .1,
    "infectious_seed_time": 50,
    # ...
}
params = project.param_set.baseline.update(calibrated_params, calibration_format=True)

In [ ]:
# run the model
model = project.run_baseline_model(params)
derived_df = model.get_derived_outputs_df()

In [ ]:
# plot the model outputs and the data used as calibration targets
target_t_min, target_t_max = project.calibration.targets[1].data.index.min(), project.calibration.targets[1].data.index.max()
all_targets = load_timeseries(os.path.join(project.get_path(), "timeseries.json"))
for target in all_targets:
    all_targets[target].index = ref_times_to_dti(REF_DATE, all_targets[target].index)  
    
pyplot.style.use("ggplot")
for output in ["infection_deaths", "cumulative_infection_deaths"]:    
    fig = pyplot.figure(figsize=(12, 8))
    axis = fig.add_subplot()

    # plot outputs
    axis = derived_df[output].plot()
    axis.set_title(output)

    # if output in project.plots:
        # all_targets[output].plot.line(ax=axis, linewidth=0., markersize=10., marker="o")
    axis.scatter(all_targets[output].index, all_targets[output], color="k", s=5, alpha=0.5, zorder=10)   
